In [1]:
import os
import cv2
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader

import pytorch_lightning as pl

from dpt.plmodels import InteriorNetDPT
from data.InteriorNetDataset import InteriorNetDataset
from util.misc import visualize_attention

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu"}')

device name: cpu


In [3]:
# k8s paths
k8s = False
k8s_repo = r'opt/repo/dynamic-inference'
k8s_pvc = r'christh9-pvc'

# path settings
model_path = 'weights/dpt_hybrid_nyu-2ce69ec7.pt'
dataset_path = 'video_inference_common/resources'
logs_path = 'train-logs'

# checkpoint_name = 'version_290'

if k8s:
    model_path = os.path.join(k8s_pvc, 'dpt-hybrid-nyu.pt')
    dataset_path = os.path.join(k8s_repo, dataset_path)
    logs_path = os.path.join(k8s_pvc, logs_path)
    
#     checkpoint_dir = os.path.join(logs_path, 'finetune', checkpoint_name, 'checkpoints')
    
    os.chdir('/')

In [4]:
model = InteriorNetDPT(batch_size=1, 
                       lr=0, 
                       num_epochs=0, 
                       model_path=model_path,
                       enable_attention_hooks=True)

model.to(device)
model.freeze()

In [5]:
dataset = InteriorNetDataset(dataset_path, split='val', 
                             transform='default', subsample=True)

loader = DataLoader(dataset, batch_size=1)

In [6]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
with torch.no_grad():
    for batch in loader:
        im, depth = batch['image'].to(device), batch['depth'].to(device)
        out = model(im).squeeze(0)
        
        visualize_attention(im, model.model, out, 'dpt_hybrid', (20,20))